<a href="https://colab.research.google.com/github/Vasundhara-Boomi/PRML-NDD-Prediction/blob/main/knn_classifier1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install tslearn


   ---------------------------------------- 374.4/374.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import LeaveOneOut

In [ ]:
# Load the transformed data
transformed_ct = pd.read_csv(r'transformed_data\transformed_ct.csv')
transformed_pd = pd.read_csv(r'transformed_data\transformed_pd.csv')
transformed_ht = pd.read_csv(r'transformed_data\transformed_ht.csv')
transformed_als = pd.read_csv(r'transformed_data\transformed_als.csv')

In [ ]:
# Add a column for labels
transformed_ct['label'] = 'control'
transformed_pd['label'] = 'parkinson'
transformed_ht['label'] = 'huntington'
transformed_als['label'] = 'als'

In [ ]:
# Combine all data into a single DataFrame
data = pd.concat([transformed_ct, transformed_pd, transformed_ht, transformed_als], ignore_index=True)

In [ ]:
data

,Unnamed: 0,Elapsed Time(s)Min,Elapsed Time(s)Max,Elapsed Time(s)Std,Elapsed Time(s)Med,Elapsed Time(s)Avg,Elapsed Time(s)Skewness,Elapsed Time(s)Kurtosis,Left Stride Interval(s)Min,Left Stride Interval(s)Max,...,Double Support Interval(s)Skewness,Double Support Interval(s)Kurtosis,Double Support Interval(%)Min,Double Support Interval(%)Max,Double Support Interval(%)Std,Double Support Interval(%)Med,Double Support Interval(%)Avg,Double Support Interval(%)Skewness,Double Support Interval(%)Kurtosis,label
0,0,21.9300,298.6000,80.541475,159.63670,159.676873,0.007791,-1.205423,0.9633,1.3967,...,1.846643,7.133888,26.69,46.03,2.582514,31.830,32.048185,1.657463,6.628575,control
1,1,22.0600,298.7500,80.301624,159.92330,160.034960,0.003471,-1.199675,0.9300,1.3533,...,6.941808,70.516428,23.18,46.80,1.989210,26.170,26.510578,5.874844,51.107931,control
2,2,21.6100,299.0567,80.423591,160.76000,160.275018,-0.002409,-1.202074,0.9667,1.3767,...,3.291091,18.503456,27.71,41.74,1.920116,31.310,31.494610,1.796461,6.547346,control
3,3,21.8933,299.7633,80.529795,160.14830,160.156720,0.005247,-1.193897,1.0033,2.0233,...,9.383892,109.145808,23.15,56.67,2.977208,26.180,26.703115,5.819219,47.517179,control
4,4,22.3600,299.4167,80.219752,159.71000,160.227956,0.012561,-1.190490,1.0300,1.2967,...,2.858954,16.734192,21.45,42.16,2.461296,25.860,26.391873,2.645490,12.970039,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,8,21.9533,288.9133,78.506061,153.10330,156.746292,-0.003560,-1.239587,1.1733,9.1833,...,5.154047,49.777404,25.00,134.31,47.483061,33.060,73.440439,0.221811,-1.928994,als
60,9,22.7167,298.4100,80.233487,161.48835,160.529298,0.003958,-1.201741,1.4233,2.1700,...,3.286439,16.668301,35.93,67.28,4.369722,43.495,44.058977,2.022438,7.497516,als
61,10,23.5267,299.4433,79.445782,159.00000,160.555286,0.022279,-1.179486,1.5067,2.2733,...,1.302906,-0.176652,23.08,161.44,43.788066,42.910,64.953711,1.311714,-0.198004,als
62,11,22.8900,299.0233,80.484751,158.15335,159.315691,0.026569,-1.205942,1.0567,1.6533,...,2.086797,6.995220,23.19,53.63,2.712919,30.210,30.731422,3.042503,21.384431,als


In [ ]:
# Drop the first column which is the index column from the CSV files
data = data.drop(columns=data.columns[0])

In [ ]:
data

,Elapsed Time(s)Min,Elapsed Time(s)Max,Elapsed Time(s)Std,Elapsed Time(s)Med,Elapsed Time(s)Avg,Elapsed Time(s)Skewness,Elapsed Time(s)Kurtosis,Left Stride Interval(s)Min,Left Stride Interval(s)Max,Left Stride Interval(s)Std,...,Double Support Interval(s)Skewness,Double Support Interval(s)Kurtosis,Double Support Interval(%)Min,Double Support Interval(%)Max,Double Support Interval(%)Std,Double Support Interval(%)Med,Double Support Interval(%)Avg,Double Support Interval(%)Skewness,Double Support Interval(%)Kurtosis,label
0,21.9300,298.6000,80.541475,159.63670,159.676873,0.007791,-1.205423,0.9633,1.3967,0.040895,...,1.846643,7.133888,26.69,46.03,2.582514,31.830,32.048185,1.657463,6.628575,control
1,22.0600,298.7500,80.301624,159.92330,160.034960,0.003471,-1.199675,0.9300,1.3533,0.041952,...,6.941808,70.516428,23.18,46.80,1.989210,26.170,26.510578,5.874844,51.107931,control
2,21.6100,299.0567,80.423591,160.76000,160.275018,-0.002409,-1.202074,0.9667,1.3767,0.036877,...,3.291091,18.503456,27.71,41.74,1.920116,31.310,31.494610,1.796461,6.547346,control
3,21.8933,299.7633,80.529795,160.14830,160.156720,0.005247,-1.193897,1.0033,2.0233,0.074304,...,9.383892,109.145808,23.15,56.67,2.977208,26.180,26.703115,5.819219,47.517179,control
4,22.3600,299.4167,80.219752,159.71000,160.227956,0.012561,-1.190490,1.0300,1.2967,0.038959,...,2.858954,16.734192,21.45,42.16,2.461296,25.860,26.391873,2.645490,12.970039,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,21.9533,288.9133,78.506061,153.10330,156.746292,-0.003560,-1.239587,1.1733,9.1833,0.561663,...,5.154047,49.777404,25.00,134.31,47.483061,33.060,73.440439,0.221811,-1.928994,als
60,22.7167,298.4100,80.233487,161.48835,160.529298,0.003958,-1.201741,1.4233,2.1700,0.102321,...,3.286439,16.668301,35.93,67.28,4.369722,43.495,44.058977,2.022438,7.497516,als
61,23.5267,299.4433,79.445782,159.00000,160.555286,0.022279,-1.179486,1.5067,2.2733,0.129713,...,1.302906,-0.176652,23.08,161.44,43.788066,42.910,64.953711,1.311714,-0.198004,als
62,22.8900,299.0233,80.484751,158.15335,159.315691,0.026569,-1.205942,1.0567,1.6533,0.073385,...,2.086797,6.995220,23.19,53.63,2.712919,30.210,30.731422,3.042503,21.384431,als


In [ ]:
# Split features and labels
X = data.drop('label', axis=1).values
y = data['label'].values

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [ ]:
X

array([[ 2.19300000e+01,  2.98600000e+02,  8.05414747e+01, ...,
         3.20481853e+01,  1.65746346e+00,  6.62857455e+00],
       [ 2.20600000e+01,  2.98750000e+02,  8.03016243e+01, ...,
         2.65105776e+01,  5.87484359e+00,  5.11079311e+01],
       [ 2.16100000e+01,  2.99056700e+02,  8.04235911e+01, ...,
         3.14946097e+01,  1.79646051e+00,  6.54734589e+00],
       ...,
       [ 2.35267000e+01,  2.99443300e+02,  7.94457820e+01, ...,
         6.49537107e+01,  1.31171373e+00, -1.98003719e-01],
       [ 2.28900000e+01,  2.99023300e+02,  8.04847509e+01, ...,
         3.07314224e+01,  3.04250326e+00,  2.13844308e+01],
       [ 2.26033000e+01,  2.99050000e+02,  8.08207567e+01, ...,
         2.83797642e+01,  1.95500813e+00,  1.52115563e+01]])

In [ ]:
y_encoded

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [ ]:

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_categorical, test_size=0.2, random_state=42)


Simple LSTM

In [ ]:
model = Sequential()
model.add(Input(shape=(X_reshaped.shape[1], X_reshaped.shape[2])))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0813 - loss: 1.3961 - val_accuracy: 0.3636 - val_loss: 1.3709
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.2771 - loss: 1.3890 - val_accuracy: 0.5455 - val_loss: 1.3701
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3958 - loss: 1.3777 - val_accuracy: 0.5455 - val_loss: 1.3689
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4188 - loss: 1.3720 - val_accuracy: 0.5455 - val_loss: 1.3677
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4833 - loss: 1.3672 - val_accuracy: 0.4545 - val_loss: 1.3655
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5167 - loss: 1.3602 - val_accuracy: 0.5455 - val_loss: 1.3632
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5437 - loss: 1.3536 - val_accuracy: 0.5455 - val_loss: 1.3606
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5333 - loss: 1.3439 - val_accuracy: 0.3636 - val_loss: 1.3577
E

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5385 - loss: 1.2280
Test Accuracy: 0.5384615659713745


Batch Normalisation (Enhanced)

In [ ]:
model = Sequential()
model.add(Input(shape=(X_reshaped.shape[1], X_reshaped.shape[2])))
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.1688 - loss: 2.0494 - val_accuracy: 0.0909 - val_loss: 1.3895
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3542 - loss: 1.4183 - val_accuracy: 0.1818 - val_loss: 1.3890
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4188 - loss: 1.2628 - val_accuracy: 0.1818 - val_loss: 1.3879
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.4833 - loss: 1.0897 - val_accuracy: 0.1818 - val_loss: 1.3869
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.6521 - loss: 0.8936 - val_accuracy: 0.1818 - val_loss: 1.3859
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.6729 - loss: 0.8788 - val_accuracy: 0.1818 - val_loss: 1.3849
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6958 - loss: 0.6310 - val_accuracy: 0.1818 - val_loss: 1.3841
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9021 - loss: 0.5429 - val_accuracy: 0.1818 - val_loss: 

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4615 - loss: 1.2230
Test Accuracy: 0.4615384638309479


Early Stopping and Class Weights?

In [ ]:
# Compute class weights to handle imbalanced data
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weights_dict = dict(enumerate(class_weights))


In [ ]:

# %%
# Define the LSTM model
model = Sequential()
model.add(Input(shape=(X_reshaped.shape[1], X_reshaped.shape[2])))
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Set callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, class_weight=class_weights_dict, callbacks=[reduce_lr, early_stopping])


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 995ms/step - accuracy: 0.1521 - loss: 1.9951 - val_accuracy: 0.1818 - val_loss: 1.3850 - learning_rate: 0.0010
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.5104 - loss: 1.2400 - val_accuracy: 0.2727 - val_loss: 1.3798 - learning_rate: 0.0010
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.4354 - loss: 1.1731 - val_accuracy: 0.1818 - val_loss: 1.3766 - learning_rate: 0.0010
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5875 - loss: 0.9406 - val_accuracy: 0.1818 - val_loss: 1.3742 - learning_rate: 0.0010
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.7229 - loss: 0.7030 - val_accuracy: 0.1818 - val_loss: 1.3710 - learning_rate: 0.0010
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7708 - loss: 0.6324 - val_accuracy: 0.2727 - val_loss: 1.3680 - learning_rate: 0.0010
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.8250 - loss: 0.6194 - val_accur

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.4615 - loss: 1.1595
Test Accuracy: 0.4615384638309479


LOOCV

In [ ]:
def create_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(LSTM(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(y_categorical.shape[1], activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:

# %%
# Initialize LOOCV
loo = LeaveOneOut()

accuracies = []

for train_index, test_index in loo.split(X_reshaped):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_train, y_test = y_categorical[train_index], y_categorical[test_index]

    model = create_model((X_reshaped.shape[1], X_reshaped.shape[2]))

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, class_weight=class_weights_dict, verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(accuracy)


In [ ]:

# %%
# Calculate and print the average accuracy
average_accuracy = np.mean(accuracies)
print(f'Average LOOCV Accuracy: {average_accuracy}')

Average LOOCV Accuracy: 0.5625


Ensemble LSTM

In [ ]:
def create_model(input_shape, num_classes=4):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(LSTM(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Set number of units to num_classes
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Initialize LOOCV
loo = LeaveOneOut()

# Initialize empty list to store model predictions
all_predictions = []

# Iterate over LOOCV splits
for train_index, test_index in loo.split(X_reshaped):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_train, y_test = y_categorical[train_index], y_categorical[test_index]

    # Create and train model
    model = create_model((X_reshaped.shape[1], X_reshaped.shape[2]), num_classes=4)  # Set num_classes to 4
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Make predictions
    predictions = model.predict(X_test)
    all_predictions.append(predictions)

# Combine predictions from all models
ensemble_predictions = np.mean(all_predictions, axis=0)



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 750ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 684ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 691ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 756ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 637ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 610ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 642ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 689ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 820ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:

# Evaluate ensemble performance
ensemble_loss, ensemble_accuracy = model.evaluate(X_test, y_test)
print(f'Ensemble Accuracy: {ensemble_accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 760ms/step - accuracy: 0.0000e+00 - loss: 1.3676
Ensemble Accuracy: 0.0


CNNLSTM

In [ ]:

# Define a function to create the LSTM model
def create_model(units=64, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(X_reshaped.shape[1], X_reshaped.shape[2])))
    model.add(LSTM(units, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_categorical.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
%pip install tensorflow==2.12.0


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device




  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl.metadata (2.5 kB)
  Using cached tensorflow_intel-2.12.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl (1.9 kB)
   -------------------------------------- 272.8/272.8 MB 297.1 kB/s eta 0:00:00
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-intel 2.16.1
    Uninstalling tensorflow-intel-2.16.1:
      Successfully uninstalled tensorflow-intel-2.16.1
  Rolling back uninstall of tensorflow-intel
  Moving to c:\users\sanju\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\scripts\import_pb_to_tensorboard.exe
   from C:\Users\Sanju\AppData\Local\Temp\pip-uninstall-9h0yvfm5\import_pb_to_tensorboard.exe
  Moving to c:\users\sanju\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\scripts\saved_model_cli.exe
   from C:\Users\San

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:

from sklearn.model_selection import RandomizedSearchCV

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [ ]:

# Create KerasClassifier for use in RandomizedSearchCV
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)

# Define the hyperparameter grid
param_grid = {
    'units': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Perform random search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, verbose=2)
random_search_result = random_search.fit(X_train, y_train)


NameError: name 'KerasClassifier' is not defined

In [ ]:

# Print results
print("Best Parameters:", random_search_result.best_params_)
print("Best Score:", random_search_result.best_score_)

# Evaluate the best model
best_model = random_search_result.best_estimator_
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

# Plotting training history
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:

# Plotting training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()
